In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dca import synth_data, cov_util, data_util
from dca.dca import DynamicalComponentsAnalysis as DCA
from sklearn.cross_decomposition import CCA, PLSSVD
import torch

from dca_research import pf_dca
from imp import reload

In [45]:
X = synth_data.gen_lorenz_data(50000)
X += np.random.randn(*X.shape) / 50.

In [46]:
X = np.concatenate([X, X[::-1]], axis=0)

In [53]:
T = 2
d = 1
model1 = DCA(T=T, d=d, n_init=5)
model1.fit(X)
print(model1.score())

tensor(3.9806, dtype=torch.float64)


In [54]:
reload(pf_dca)
model2 = pf_dca.PastFutureDynamicalComponentsAnalysis(T=T, d=d, n_init=5)
model2.fit(X)
print(model2.score())

tensor(3.9806, dtype=torch.float64)


In [55]:
model1.coef_

array([[0.80849845],
       [0.58801323],
       [0.02388927]])

In [56]:
model2.coef_[0]

array([[0.80873483],
       [0.5877508 ],
       [0.02229311]])

In [57]:
model2.coef_[1]

array([[0.80874543],
       [0.58773699],
       [0.02227242]])

In [16]:
model3 = CCA(n_components=d, scale=False)
model3.fit(X[:-1], X[1:])
model4 = pf_dca.PastFutureDynamicalComponentsAnalysis(T=1, d=model3.n_components, verbose=True, n_init=5)
model4.estimate_cross_covariance(X)
model4.coef_ = (model3.x_weights_, model3.y_weights_)
print(model4.score())

tensor(3.5796, dtype=torch.float64)


In [ ]:
print(model3.x_weights_)
print(model3.y_weights_)

In [ ]:
print(model2.coef_[0])
print(model2.coef_[1])

In [ ]:
plt.imshow(cov_util.calc_cov_from_cross_cov_mats(model2.cross_covs.numpy()))

In [ ]:
np.linalg.eigvalsh(cov_util.calc_cov_from_cross_cov_mats(cov_util.project_cross_cov_mats(model2.cross_covs.numpy(), proj=np.random.randn(3, 1))))

In [ ]:
reload(pf_dca)
dp = 2
df = 3
T = 3
proj = np.random.randn(3, dp)
proj2 = np.random.randn(3, df)
#proj2 = proj
ccms = cov_util.calc_cross_cov_mats_from_data(X, T=T*2)
bigcov = cov_util.calc_cov_from_cross_cov_mats(ccms)
cov = pf_dca.calc_cov_from_cross_cov_mats(*pf_dca.project_cross_cov_mats(ccms,
                                          proj,
                                          proj2))
tccms = torch.tensor(ccms)
tcov = pf_dca.calc_cov_from_cross_cov_mats(*pf_dca.project_cross_cov_mats(tccms,
                                            torch.tensor(proj),
                                            torch.tensor(proj2)))
Xl = data_util.form_lag_matrix(X, T=T*2)
big_proj = np.zeros((2*T*3, T*(dp+df)))
for ii in range(T):
    big_proj[ii*3:(ii+1)*3, ii*dp:(ii+1)*dp] = proj
    big_proj[(T+ii)*3:(T+ii+1)*3, T*dp+ii*df:T*dp+(ii+1)*df] = proj2
bigcov2 = np.cov(Xl, rowvar=False)
cov2 = np.cov(Xl.dot(big_proj), rowvar=False)
#print(np.linalg.eigvalsh(bigcov))
#print(np.linalg.eigvalsh(cov))
#print(np.linalg.eigvalsh(cov[:T*dp, :T*dp]))
#print(np.linalg.eigvalsh(cov[T*df:, T*df:]))

In [ ]:
plt.scatter(tcov.numpy().ravel(), cov.ravel())

In [ ]:
for ii in range(T):
    mat = cov2[:dp, ii*dp:(ii+1)*dp]
    print(mat)
    print(mat.shape)
for ii in range(T):
    mat = cov2[:dp, T*dp+ii*df:T*dp+(ii+1)*df]
    print(mat)
    print(mat.shape)

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(bigcov.ravel(), bigcov2.ravel(), marker='.')
plt.figure(figsize=(2, 2))
plt.scatter(cov.ravel(), cov2.ravel(), marker='.')
plt.figure(figsize=(2, 2))
plt.scatter(cov[:dp*T,:dp*T].ravel(), cov2[:dp*T,:dp*T].ravel(), marker='.')

In [ ]:
plt.imshow(cov, vmin=min(cov.min(), cov2.min()), vmax=max(cov.max(), cov2.max()))

In [ ]:
plt.imshow(cov2, vmin=min(cov.min(), cov2.min()), vmax=max(cov.max(), cov2.max()))

In [ ]:
cov[:T*d, T*d:]

In [ ]:
cov2[:T*d, T*d:]

In [ ]:
cpp, cff, cpf = pf_dca.project_cross_cov_mats(ccms, proj, proj2)

In [ ]:
np.linalg.eigvalsh(cov_util.calc_cov_from_cross_cov_mats(cpp))

In [ ]:
np.allclose(cov[T*d:, :T*d], cov[:T*d, T*d:].T)